In [2]:
#基于协同过滤的推荐算法（Collaborative Filtering）
#User-Based CF 基于用户的协同过滤推荐

import numpy as np
import pandas as pd

users = ['User1', 'User2', 'User3', 'User4', 'User5']
items = ['ItemA', 'ItemB', 'ItemC', 'ItemD', 'ItemE']

# 1.构造用户购买记录数据集
datasets = [
    [1,0,1,1,0],
    [1,0,0,1,1],
    [1,0,1,0,0],
    [0,1,0,1,1],
    [1,1,1,0,1],
]
df = pd.DataFrame(datasets, columns=items, index=users)
print(df)

# 2.计算用户间相似度
user_similar = []
for user in df.index:
    # 计算每个用户和其他用户间的相似度
    similarity_list = []
    for other_user in df.index:
        user_list = list(df.loc[user])
        other_user_list = list(df.loc[other_user])
        common_num = sum([ 1 for i in range(len(user_list)) if user_list[i] + other_user_list[i] == 2 ])
        similarity = (common_num / sum(user_list)) * (common_num / sum(other_user_list))
        similarity_list.append(similarity)
    # 构建用户间相似度矩阵
    user_similar.append(similarity_list)

user_similar = pd.DataFrame(user_similar, columns=users, index=users)

print("用户之间的两两相似度：")
print(user_similar)

# 3.构造每个用户相似度topN的用户字典
topN_users = {}
for i in user_similar.index:
    # 取出每一列数据，并删除自身，然后排序数据
    _df = user_similar.loc[i].drop([i])
    _df_sorted = _df.sort_values(ascending=False)

    top2 = list(_df_sorted.index[:2])
    topN_users[i] = top2

print("Top2相似用户：")
print(topN_users)

# 4.构建推荐结果
rs_results = {}
for user, sim_users in topN_users.items():
    # 存储每个用户推荐结果
    rs_result = set()   
    # 找到最相似的topN用户购买过的物品
    for sim_user in sim_users:
        rs_result = rs_result.union(set(df.loc[sim_user].replace(0, np.nan).dropna().index))
    # 过滤掉已经购买过的物品
    rs_result -= set(df.loc[user].replace(0, np.nan).dropna().index)

    rs_results[user] = rs_result

print("最终推荐结果：")
print(rs_results)


       ItemA  ItemB  ItemC  ItemD  ItemE
User1      1      0      1      1      0
User2      1      0      0      1      1
User3      1      0      1      0      0
User4      0      1      0      1      1
User5      1      1      1      0      1
用户之间的两两相似度：
          User1     User2     User3     User4     User5
User1  1.000000  0.444444  0.666667  0.111111  0.333333
User2  0.444444  1.000000  0.166667  0.444444  0.333333
User3  0.666667  0.166667  1.000000  0.000000  0.500000
User4  0.111111  0.444444  0.000000  1.000000  0.333333
User5  0.333333  0.333333  0.500000  0.333333  1.000000
Top2相似用户：
{'User1': ['User3', 'User2'], 'User2': ['User1', 'User4'], 'User3': ['User1', 'User5'], 'User4': ['User2', 'User5'], 'User5': ['User3', 'User1']}
最终推荐结果：
{'User1': {'ItemE'}, 'User2': {'ItemB', 'ItemC'}, 'User3': {'ItemB', 'ItemE', 'ItemD'}, 'User4': {'ItemA', 'ItemC'}, 'User5': {'ItemD'}}


In [ ]:
#Item-Based CF 基于物品的协同过滤推荐
